In [21]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="6"

In [56]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from src import milan, milannotations

In [23]:
alexnet_imagenet = milannotations.load('alexnet/imagenet')

load alexnet/imagenet: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.88s/it]


In [ ]:
alexnet_imagenet.samples_by_layer_unit[('conv4', 113)].annotations

In [81]:
key_lst = [key for key in alexnet_imagenet.samples_by_layer_unit.keys() if key[0] == 'conv5']
sorted(key_lst, key = lambda x: x[1])

[('conv5', 0),
 ('conv5', 1),
 ('conv5', 2),
 ('conv5', 3),
 ('conv5', 4),
 ('conv5', 5),
 ('conv5', 6),
 ('conv5', 7),
 ('conv5', 8),
 ('conv5', 9),
 ('conv5', 10),
 ('conv5', 11),
 ('conv5', 12),
 ('conv5', 13),
 ('conv5', 14),
 ('conv5', 15),
 ('conv5', 16),
 ('conv5', 17),
 ('conv5', 18),
 ('conv5', 19),
 ('conv5', 20),
 ('conv5', 21),
 ('conv5', 22),
 ('conv5', 23),
 ('conv5', 24),
 ('conv5', 25),
 ('conv5', 26),
 ('conv5', 27),
 ('conv5', 28),
 ('conv5', 29),
 ('conv5', 30),
 ('conv5', 31),
 ('conv5', 32),
 ('conv5', 33),
 ('conv5', 34),
 ('conv5', 35),
 ('conv5', 36),
 ('conv5', 37),
 ('conv5', 38),
 ('conv5', 39),
 ('conv5', 40),
 ('conv5', 41),
 ('conv5', 42),
 ('conv5', 43),
 ('conv5', 44),
 ('conv5', 45),
 ('conv5', 46),
 ('conv5', 47),
 ('conv5', 48),
 ('conv5', 49),
 ('conv5', 50),
 ('conv5', 51),
 ('conv5', 52),
 ('conv5', 53),
 ('conv5', 54),
 ('conv5', 55),
 ('conv5', 56),
 ('conv5', 57),
 ('conv5', 58),
 ('conv5', 59),
 ('conv5', 60),
 ('conv5', 61),
 ('conv5', 62),
 (

In [ ]:
decoder = milan.pretrained('alexnet')

In [10]:
from src.exemplars.models import default_model_configs
from src.exemplars.models import load

In [8]:
default_model_configs()['alexnet/imagenet'].__dict__

{'factory': <function src.deps.ext.torchvision.models.alexnet_seq(**kwargs: Any) -> torch.nn.modules.container.Sequential>,
 'defaults': {'pretrained': True},
 'url': None,
 'requires_path': False,
 'load_weights': False,
 'transform_weights': None,
 'layers': ['conv1', 'conv2', 'conv3', 'conv4', 'conv5'],
 'exemplars': ModelExemplarsConfig(k=None, quantile=None, output_size=None, batch_size=None, image_size=None, renormalizer=None)}

In [133]:
model, layer, config = load('alexnet/imagenet')

In [33]:
X_test = torch.randn(1, 3, 224, 224)
model(X_test).shape

torch.Size([1, 1000])

In [39]:
x = torch.randn(10, 1000, requires_grad=True)
out = torch.index_select(x, 1, torch.tensor([0, 2]))

In [40]:
out.shape

torch.Size([10, 2])

In [116]:
model

Sequential(
  (conv1): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (relu1): ReLU(inplace=True)
  (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU(inplace=True)
  (pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3): ReLU(inplace=True)
  (conv4): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu4): ReLU(inplace=True)
  (conv5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu5): ReLU(inplace=True)
  (pool5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (dropout6): Dropout(p=0.5, inplace=False)
  (fc6): Linear(in_features=9216, out_features

In [134]:
def multiply_mask(layer, mask):
    layer_mask_idx = {
        'conv1': torch.arange(0, 64),
        'conv2': torch.arange(64, 256),
        'conv3': torch.arange(256, 640),
        'conv4': torch.arange(640, 896),
        'conv5': torch.arange(876, 1132)
    }
    layer_mask = torch.index_select(mask, 1, layer_mask_idx[layer])
    layer_mask = layer_mask[:, :, None, None]
    def hook(model, input, output):
        print(output.shape)
        print(output * layer_mask)
        return output * layer_mask
    return hook

    
    
mask = torch.zeros(1, 1132)
layer_hooks = [None] * 5
for hook_idx, layer_idx, layer_name in zip(range(5), [0, 3, 6, 8, 10], ['conv1', 'conv2', 'conv3', 'conv4', 'conv5']):
    layer_hook = model[layer_idx].register_forward_hook(multiply_mask(layer_name, mask))
    layer_hooks[hook_idx] = layer_hook
    


output = model(X_test)
print(output)

for layer_hook in layer_hooks:
    layer_hook.remove()

torch.Size([1, 64, 55, 55])
tensor([[[[-0., -0., 0.,  ..., -0., -0., 0.],
          [-0., 0., -0.,  ..., 0., 0., -0.],
          [-0., 0., -0.,  ..., -0., -0., -0.],
          ...,
          [-0., 0., -0.,  ..., -0., 0., 0.],
          [-0., -0., 0.,  ..., 0., -0., -0.],
          [-0., -0., -0.,  ..., -0., -0., -0.]],

         [[-0., -0., -0.,  ..., -0., -0., -0.],
          [-0., -0., -0.,  ..., -0., -0., -0.],
          [-0., -0., -0.,  ..., -0., -0., -0.],
          ...,
          [-0., -0., -0.,  ..., -0., -0., -0.],
          [-0., -0., -0.,  ..., -0., -0., -0.],
          [-0., -0., -0.,  ..., -0., -0., -0.]],

         [[0., -0., -0.,  ..., 0., 0., -0.],
          [0., -0., -0.,  ..., -0., 0., 0.],
          [-0., -0., -0.,  ..., -0., -0., -0.],
          ...,
          [0., -0., 0.,  ..., 0., -0., 0.],
          [0., -0., 0.,  ..., 0., -0., -0.],
          [-0., -0., -0.,  ..., 0., -0., -0.]],

         ...,

         [[0., 0., -0.,  ..., -0., -0., -0.],
          [0., 0., 0.

In [132]:
X_test

tensor([[[[ 7.0384e-01,  9.0024e-01, -2.6086e-01,  ..., -5.4962e-01,
            7.5990e-01,  1.2810e+00],
          [ 8.3528e-01, -1.2034e+00,  2.7475e+00,  ..., -1.2221e+00,
            4.3180e-01, -6.8186e-01],
          [-3.5282e-01,  3.6149e-01, -1.5244e+00,  ...,  5.0769e-01,
           -1.7284e+00, -1.2043e-01],
          ...,
          [ 2.4861e+00,  5.0340e-01, -1.7340e-03,  ..., -1.2344e+00,
           -1.5609e+00, -1.0391e+00],
          [ 1.3990e+00,  2.5919e+00, -1.4409e+00,  ...,  7.7594e-01,
            1.4985e-01,  1.2011e+00],
          [-2.4594e+00,  1.2059e-01, -1.3795e-01,  ...,  4.8951e-01,
           -8.8356e-01,  1.9623e-01]],

         [[-1.0343e+00,  1.0651e+00, -2.0574e-02,  ...,  2.8781e-01,
            4.8236e-01, -1.7541e+00],
          [-1.6156e+00, -1.6899e+00, -3.0753e-01,  ...,  6.2082e-01,
            1.0829e+00, -6.4283e-01],
          [ 2.5539e-01, -1.4353e+00,  4.6495e-01,  ..., -8.3487e-01,
           -3.9261e-01, -9.6195e-01],
          ...,
     